<a href="https://colab.research.google.com/github/Atabak-Touri/NLP-CNN_RNN/blob/main/sentenceclassification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import collections
import math
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf

seed = 54321

%env TF_FORCE_GPU_ALLOW_GROWTH=true

env: TF_FORCE_GPU_ALLOW_GROWTH=true


**Downloading the dataset**

In [2]:
url = 'http://cogcomp.org/Data/QA/QC/'
dir_name = 'data'

def download_data(dir_name, filename, expected_bytes):
    os.makedirs(dir_name, exist_ok=True)
    if not os.path.exists(os.path.join(dir_name,filename)):
        filepath, _ = urlretrieve(url + filename, os.path.join(dir_name,filename))
    else:
        filepath = os.path.join(dir_name, filename)

    statinfo = os.stat(filepath)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filepath)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filepath + '. Can you get to it with a browser?')

    return filepath

train_filename = download_data(dir_name, 'train_5500.label', 335858)
test_filename = download_data(dir_name, 'TREC_10.label',23354)

Found and verified data/train_5500.label
Found and verified data/TREC_10.label


going through each line and split the question, category and sub-category into lists.
in the end we have splitted testing and training lists.

In [5]:
def read_data(filename):
    '''
    Read data from a file with given filename
    Returns a list of strings where each string is a lower case word
    '''

    # Holds question strings, categories and sub categories
    # category/sub_cateory definitions: https://cogcomp.seas.upenn.edu/Data/QA/QC/definition.html
    questions, categories, sub_categories = [], [], []

    with open(filename,'r',encoding='latin-1') as f:
        # Read each line
        for row in f:
            # Each string has format <cat>:<sub cat> <question>
            # Split by : to separate cat and (sub_cat + question)
            row_str = row.split(":")
            cat, sub_cat_and_question = row_str[0], row_str[1]
            tokens = sub_cat_and_question.split(' ')
            # The first word in sub_cat_and_question is the sub category
            # rest is the question
            sub_cat, question = tokens[0], ' '.join(tokens[1:])

            questions.append(question.lower().strip())
            categories.append(cat)
            sub_categories.append(sub_cat)


    return questions, categories, sub_categories

train_questions, train_categories, train_sub_categories = read_data(train_filename)
test_questions, test_categories, test_sub_categories = read_data(test_filename)

n_samples = 10
print(f"train_questions has {len(train_questions)} questions / {len(train_categories)} labels")
print("Some samples")
for question, cat, sub_cat in zip(train_questions[:n_samples], train_categories[:n_samples], train_sub_categories[:n_samples]):
    print(f"\t{question} / cat - {cat} / sub_cat - {sub_cat}")

print(f"\ntest_questions has {len(test_questions)} questions / {len(test_categories)} labels")
print("Some samples")
for question, cat, sub_cat in zip(test_questions[:n_samples], test_categories[:n_samples], test_sub_categories[:n_samples]):
    print(f"\t{question} / cat - {cat} / sub_cat - {sub_cat}")

train_questions has 5452 questions / 5452 labels
Some samples
	how did serfdom develop in and then leave russia ? / cat - DESC / sub_cat - manner
	what films featured the character popeye doyle ? / cat - ENTY / sub_cat - cremat
	how can i find a list of celebrities ' real names ? / cat - DESC / sub_cat - manner
	what fowl grabs the spotlight after the chinese year of the monkey ? / cat - ENTY / sub_cat - animal
	what is the full form of .com ? / cat - ABBR / sub_cat - exp
	what contemptible scoundrel stole the cork from my lunch ? / cat - HUM / sub_cat - ind
	what team did baseball 's st. louis browns become ? / cat - HUM / sub_cat - gr
	what is the oldest profession ? / cat - HUM / sub_cat - title
	what are liver enzymes ? / cat - DESC / sub_cat - def
	name the scar-faced bounty hunter of the old west . / cat - HUM / sub_cat - ind

test_questions has 500 questions / 500 labels
Some samples
	how far is it from denver to aspen ? / cat - NUM / sub_cat - dist
	what county is modesto , cal

**Creating Pandas data frame**

pandas dataframe is expressive frames for storing multi dimensional data.

In [6]:
train_df = pd.DataFrame(
    {'question': train_questions, 'category': train_categories, 'sub_category': train_sub_categories}
)
test_df = pd.DataFrame(
    {'question': test_questions, 'category': test_categories, 'sub_category': test_sub_categories}
)

train_df.head(n=10)

,question,category,sub_category
0,how did serfdom develop in and then leave russ...,DESC,manner
1,what films featured the character popeye doyle ?,ENTY,cremat
2,how can i find a list of celebrities ' real na...,DESC,manner
3,what fowl grabs the spotlight after the chines...,ENTY,animal
4,what is the full form of .com ?,ABBR,exp
5,what contemptible scoundrel stole the cork fro...,HUM,ind
6,what team did baseball 's st. louis browns bec...,HUM,gr
7,what is the oldest profession ?,HUM,title
8,what are liver enzymes ?,DESC,def
9,name the scar-faced bounty hunter of the old w...,HUM,ind
